In [94]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from yullab.models import MLP, CNN, RNN, LSTM
from yullab.datasets import FashionDataset
from yullab.trainer import train, test
from time import time
import getpass

In [95]:
username = getpass.getuser()
fashion_train_filename = f'/Users/{username}/data/fashion_mnist/fashion-mnist_train.csv'
fashion_val_filename = f'/Users/{username}/data/fashion_mnist/fashion-mnist_test.csv'

text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [96]:
# for cnn, mlp
start_time = time()
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(0, 1)])

train_dataset = FashionDataset(fashion_train_filename, train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = FashionDataset(fashion_val_filename, train_transforms)
val_loader = DataLoader(val_dataset,
                        batch_size=len(val_dataset),
                        shuffle=False)
# net = MLP()
#net = CNN()
net = LSTM()

lr = 0.005
n_epochs = 3

optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train_losses = train(train_loader, net, criterion, optimizer, n_epochs)
test_loss, accuracy  = test(val_loader, net, criterion)
end_time = time()
print(end_time-start_time)

Epoch: 1/3..  Training Loss: 0.617.. 
Epoch: 2/3..  Training Loss: 0.381.. 
Epoch: 3/3..  Training Loss: 0.334.. 
Test Loss: 0.324..  Test Accuracy: 0.881
217.67272424697876
